In [7]:
import pandas as pd
import json

In [ ]:
# Define which M49 codes to use (removed regions like "Europe", only kept indivisual countries)
M49 = [108,174,262,232,231,404,450,454,480,175,508,638,646,690,706,728,800,834,894,716,24,120,140,148,178,180,226,266,678,12,818,434,504,729,788,732,72,748,426,516,710,204,854,132,384,270,288,324,624,430,466,478,562,566,654,686,694,768,398,417,762,795,860,156,344,446,158,408,392,496,410,96,116,360,418,458,104,608,702,764,626,704,4,50,64,356,364,462,524,586,144,51,31,48,196,268,368,376,400,414,422,512,634,682,275,760,792,784,887,112,100,203,348,616,498,642,643,703,804,830,208,233,234,246,352,372,833,428,440,578,752,826,8,20,70,191,292,300,336,380,470,499,807,620,674,688,705,724,40,56,250,276,438,442,492,528,756,660,28,533,44,52,535,92,136,192,531,212,214,308,312,332,388,474,500,630,652,659,662,663,670,534,780,796,850,84,188,222,320,340,484,558,591,32,68,76,152,170,218,238,254,328,600,604,740,858,862,60,124,304,666,840,36,554,242,540,598,90,548,316,296,584,583,520,580,585,16,184,258,570,882,772,776,798,876]
# If I would like to filter out only the important values
# mask = df['Subscription'].isin(active_statuses)

In [9]:
# Data from "undesa_pd_2020_ims_stock_by_sex_and_destination"
# Read excel file
xls = pd.ExcelFile(
    '../data_world/undesa_pd_2020_ims_stock_by_sex_and_destination.xlsx')
# Names, M49 location codes and how the data was estimated 
dfNamesCodesTypes = pd.read_excel(xls, 'Table 1', header=10, usecols="B, D, E")
# Formatting 
dfNamesCodesTypes["Region, development group, country or area"] = dfNamesCodesTypes["Region, development group, country or area"].str.strip().str.rstrip('*')
dfNamesCodesTypes["Type of data"].fillna('NaN', inplace=True)
# df1 --> "Table 1" --> "International migrant stock at mid-year, both sexes combined"
# df2 --> "Table 2" --> "Total population at mid-year, both sexes combined (thousands)"
# df3 --> "Table 3" --> "International migrant stock as a percentage of the total population, both sexes combined"
# df4 --> "Table 5" --> "Annual rate of change of the migrant stock, both sexes combined"
# df5 --> "Table 6" --> "Refugee and asylum seekers at mid-year, both sexes combined"
# df6 --> "Table 6" --> "Refugee and asylum seekers as a percentage of the international migrant stock"
# df7 --> "Table 6" --> "Annual rate of change of the refugee and asylum seekers, both sexes combined"
df1 = pd.read_excel(xls, 'Table 1', header=10, usecols="F:L")
df2 = pd.read_excel(xls, 'Table 2', header=10, usecols="F:L")
df3 = pd.read_excel(xls, 'Table 3', header=10, usecols="F:L")
df4 = pd.read_excel(xls, 'Table 5', header=10, usecols="F:K")
df5 = pd.read_excel(xls, 'Table 6', header=10, usecols="F:L")
df6 = pd.read_excel(xls, 'Table 6', header=10, usecols="M:S")
df7 = pd.read_excel(xls, 'Table 6', header=10, usecols="T:Y")

data_dtfs = [df1, df2, df3, df4, df5, df6, df7]
descriptions = ["International migrant stock at mid-year, both sexes combined",
                "Total population at mid-year, both sexes combined (thousands)",
                "International migrant stock as a percentage of the total population, both sexes combined",
                "Annual rate of change of the migrant stock, both sexes combined",
                "Refugee and asylum seekers at mid-year, both sexes combined",
                "Refugee and asylum seekers as a percentage of the international migrant stock",
                "Annual rate of change of the refugee and asylum seekers, both sexes combined"]

# Data structure:
# The location codes and orders are the same for all tables
# Since the M49 codes ("code") and names of the country/region/... names ("names") are always
# the same order for each table they only appear once in the beginning
# Below the envisionned structure of the json
# {"name": [...], "code": [...], "type": [...], "data": [{"description": "...", "data": {"1995": [...], ...}}, ...]}
data_undesa = {
    "name": dfNamesCodesTypes["Region, development group, country or area"].tolist(),
    "code": dfNamesCodesTypes["Location code"].tolist(),
    "type": dfNamesCodesTypes["Type of data"].tolist(),
    "data": [{"description": descriptions[i], "data": data_dtfs[i].to_dict(orient='list')} for i in range(len(data_dtfs))]
}

# print(dfNamesCodesTypes)
# print(list(df1))
# print(df7.to_dict(orient='list'))
# print([{"description": descriptions[i], "data": data_dtfs[i].to_dict(orient='list')} for i in range(len(data_dtfs))])

In [8]:
with open("../../website/data/undesa_data.json", "w") as outfile:
    json.dump(data_undesa, outfile)